In [ ]:
import mysql.connector
from mysql.connector import Error
from hdfs.ext.kerberos import KerberosClient

import pandas as pd

#initializing hdfs client
hdfs_client = KerberosClient('http://master-node:9870','OPTIONAL')

In [ ]:
# ###SQL Connections
# try:
#      # Establishing the connection to MySQL server (without specifying a database)
#         connection = mysql.connector.connect(host='127.0.0.1',
#                                              port=3309,
#                                              database='webscrape',
#                                              user='root',
#                                              password='password',
#                                              auth_plugin='mysql_native_password')
#         if connection.is_connected():
#             db_Info = connection.get_server_info()
#             print("Connected to MySQL Server version ", db_Info)
#             cursor = connection.cursor()
#             cursor.execute("select database();")
#             record = cursor.fetchone()
#             print("You're connected to database: ", record)
            
#             # print(connection)
#         cursor = connection.cursor()
#         table_creation_query = """CREATE TABLE IF NOT EXISTS crypto_currency (
#                                   id INT AUTO_INCREMENT,
#                                   symbol VARCHAR(50),
#                                   name VARCHAR(255),
#                                   priceintraday varchar(50),
#                                   changevalue varchar(50),
#                                   percentchange varchar(50),
#                                   marketcap VARCHAR(50),
#                                   volumeincurrencysinceutc varchar(50),
#                                   volumeincurrency24hr varchar(50),
#                                   totalvolumeallcurrencies24hr varchar(50),
#                                   circulatingsupply varchar(15), PRIMARY KEY (id));"""
#         cursor.execute(table_creation_query)
        
#         ##Inserting data into the tables
#         try:
#             for _, row in df.iterrows():
#                 insert_query = "INSERT INTO crypto_currency(symbol,name,priceintraday,changevalue,percentchange,marketcap,volumeincurrencysinceutc,volumeincurrency24hr,totalvolumeallcurrencies24hr,circulatingsupply) VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
#                 print(row)
#                 cursor.execute(insert_query, tuple(row))
#              #Insert the rows   
            
#             print("Inserted financial_data")
#             ##commit the db values
#             connection.commit()
#             # print()
#         except Error as e:
#             print(e)
# except Error as e:       
#            print(e) 
# finally:
#         if connection.is_connected():
#             cursor.close()
#             connection.close()
#             print("MySQL connection is closed")

In [ ]:
try:
         # Establishing the connection to MySQL server (without specifying a database)
        connection = mysql.connector.connect(host='127.0.0.1',
                                             port=3309,
                                             database='webscrape',
                                             user='root',
                                             password='password',
                                             auth_plugin='mysql_native_password')
        if connection.is_connected():
            db_Info = connection.get_server_info()
            print("Connected to MySQL Server version ", db_Info)
            cursor = connection.cursor()
            cursor.execute("select database();")
            record = cursor.fetchone()
            print("You're connected to database: ", record)
            
            # print(connection)
        cursor = connection.cursor()
        # List of table creation queries
        table_creation_queries = ["""CREATE TABLE IF NOT EXISTS crypto_currency (
                                  id INT AUTO_INCREMENT,
                                  symbol VARCHAR(50),
                                  name VARCHAR(255),
                                  priceintraday varchar(50),
                                  changevalue varchar(50),
                                  percentchange varchar(50),
                                  marketcap VARCHAR(50),
                                  volumeincurrencysinceutc varchar(50),
                                  volumeincurrency24hr varchar(50),
                                  totalvolumeallcurrencies24hr varchar(50),
                                  circulatingsupply varchar(15), PRIMARY KEY (id));""",
                                  """CREATE TABLE IF NOT EXISTS currencies (id INT AUTO_INCREMENT,currencypairname VARCHAR(50),currencyexchangevalue DECIMAL(10, 5),currencyexchangechangevalue DECIMAL(10, 5),currencyexchangechangepercentage VARCHAR(10),PRIMARY KEY (id));""",
                                  """CREATE TABLE IF NOT EXISTS indices (id INT AUTO_INCREMENT,indexname VARCHAR(120),indexvalue DECIMAL(30, 10),indexchangevalue DECIMAL(30, 10),indexchangepercentage VARCHAR(10),indexregion varchar(50),PRIMARY KEY (id));""",                                  
                                  """CREATE TABLE IF NOT EXISTS stocks (id INT AUTO_INCREMENT,stockname VARCHAR(120),stockvalue VARCHAR(55),stockchangevalue VARCHAR(55),stockchangepercentage VARCHAR(10),stocklabel varchar(50),PRIMARY KEY (id));""",
                                  """CREATE TABLE IF NOT EXISTS cryptocurrencies (id INT AUTO_INCREMENT,cryptoname VARCHAR(150),cryptovalue DECIMAL(30, 10),cryptochangevalue DECIMAL(30, 10),cryptochangepercentage VARCHAR(10),PRIMARY KEY (id));"""]

        # ## execute the above array to create tables
        for query in table_creation_queries:
            cursor.execute(query)
            print(query)
            print("Executed a table creation query.")
       
       ##Inserting data into the tables
        try: 
            with hdfs_client.read('/data/crypto-currencies.csv') as reader_crypto_curr:
                df2_crypto_currency = pd.read_csv(reader_crypto_curr, sep=',')
            for _, row in df2_crypto_currency.iterrows():
                insert_query = "INSERT INTO crypto_currency(symbol,name,priceintraday,changevalue,percentchange,marketcap,volumeincurrencysinceutc,volumeincurrency24hr,totalvolumeallcurrencies24hr,circulatingsupply) VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
                cursor.execute(insert_query, tuple(row))
            print("Inserted currencies")
                
                
            with hdfs_client.read('/data/currencies.csv') as reader_curr:
                df2_currencies = pd.read_csv(reader_curr, sep=',')
            for _, row in df2_currencies.iterrows():
                insert_query = "INSERT INTO currencies (currencypairname,currencyexchangevalue,currencyexchangechangevalue,currencyexchangechangepercentage) VALUES (%s, %s, %s,%s)"
                cursor.execute(insert_query, tuple(row))
            print("Inserted currencies")
            
            with hdfs_client.read('/data/crypto-currencies.csv') as reader_curr:
                df2_crypto_currencies = pd.read_csv(reader_curr, sep=',')
            for _, row in df2_crypto_currencies.iterrows():
                insert_query = "INSERT INTO cryptocurrencies (cryptoname,cryptovalue,cryptochangevalue,cryptochangepercentage) VALUES (%s, %s, %s,%s)"
                cursor.execute(insert_query, tuple(row))
            print("Inserted crypto-currencies")
                
            
            with hdfs_client.read('/data/american-indices.csv') as reader_curr:
                df2_american_indices = pd.read_csv(reader_curr, sep=',')   
            for _, row in df2_american_indices.iterrows():
                insert_query = "INSERT INTO indices (indexname,indexvalue,indexchangevalue,indexchangepercentage,indexregion) VALUES (%s, %s, %s,%s,'american')"
                cursor.execute(insert_query, tuple(row))
            print("Inserted american")
            
            
                
            with hdfs_client.read('/data/asian-indices.csv') as reader_curr:
                df2_asian_indices = pd.read_csv(reader_curr, sep=',')
            for _, row in df2_asian_indices.iterrows():
                insert_query = "INSERT INTO indices (indexname,indexvalue,indexchangevalue,indexchangepercentage,indexregion) VALUES (%s, %s, %s,%s,'asian')"
                cursor.execute(insert_query, tuple(row))
            print("Inserted asian")
        
            with hdfs_client.read('/data/emea-indices.csv') as reader_curr:
                df2_emea_indices = pd.read_csv(reader_curr, sep=',')
            for _, row in df2_emea_indices.iterrows():
                insert_query = "INSERT INTO indices (indexname,indexvalue,indexchangevalue,indexchangepercentage,indexregion) VALUES (%s, %s, %s,%s,'emea')"
                cursor.execute(insert_query, tuple(row))
            print("Inserted emea")
            
            with hdfs_client.read('/data/looser-stocks.csv') as reader_curr:
                df2_looser_stocks = pd.read_csv(reader_curr, sep=',')
            for _, row in df2_looser_stocks.iterrows():
                insert_query = "INSERT INTO stocks (stockname,stockvalue,stockchangevalue,stockchangepercentage,stocklabel) VALUES (%s, %s, %s,%s,'looser')"
                cursor.execute(insert_query, tuple(row))
            print("Inserted loosers")
            
            with hdfs_client.read('/data/gainer-stocks.csv') as reader_curr:
                df2_gainer_stocks = pd.read_csv(reader_curr, sep=',')
            for _, row in df2_gainer_stocks.iterrows():
                insert_query = "INSERT INTO stocks (stockname,stockvalue,stockchangevalue,stockchangepercentage,stocklabel) VALUES (%s, %s, %s,%s,'gainer')"
                cursor.execute(insert_query, tuple(row))
            print("Inserted gainer")
            
            with hdfs_client.read('/data/climateleaders.csv') as reader_curr:
                df2_climateLeaders = pd.read_csv(reader_curr, sep=',')
            for _, row in df2_climateLeaders.iterrows():
                insert_query = "INSERT INTO stocks (stockname,stockvalue,stockchangevalue,stockchangepercentage,stocklabel) VALUES (%s, %s, %s,%s,'climate leader')"
                cursor.execute(insert_query, tuple(row))
            print("Inserted climate")
            
            ##commit the db values
            connection.commit()
        except Error as e:
            print(e)
        # print(df_crypto)
except Error as e:
        print()
finally:
        if connection.is_connected():
            cursor.close()
            connection.close()
            print("MySQL connection is closed")